In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import pickle
from copy import deepcopy
from read.read_functions import *
from read.pickle_functions import *
from create_all_users import create_all_users
from create_rating_statistic import create_ratings_stat

pd.set_option('display.max_columns',40)
%load_ext autoreload
%autoreload 2

In [4]:
beer_advocate_path = '../datas/BeerAdvocate/pickle/'
matched_beer_data_path = '../datas/matched_beer_data/pickle'
rate_beer_path = '../datas/RateBeer/pickle'

advocate_beers_path_pickle = beer_advocate_path + 'beers.csv'
advovate_breweries_path_pickle = beer_advocate_path + 'breweries.csv'
advocate_ratings_path_pickle = beer_advocate_path + 'ratings.txt'
advocate_reviews_path_pickle = beer_advocate_path + 'reviews.txt'
advocate_users_path_pickle = beer_advocate_path + 'users.csv'

matched_beer_beers_pickle = matched_beer_data_path + 'beers.csv'
matched_beer_breweries_pickle = matched_beer_data_path + 'breweries.csv'
matched_beer_ratings_ba_pickle = matched_beer_data_path + 'ratings_ba.txt'
matched_beer_ratings_rb_pickle = matched_beer_data_path + 'ratings_rb.txt'
matched_beer_ratings_pickle = matched_beer_data_path + 'ratings.csv'
matched_beer_users_approx_pickle = matched_beer_data_path + 'users_approx.csv'
matched_beer_users_pickle = matched_beer_data_path + 'users.csv'

rate_beer_beers_pickle = rate_beer_path + 'beers.csv'
rate_beer_breweries_pickle = rate_beer_path + 'breweries.csv'
rate_beer_users_pickle = rate_beer_path + 'users.csv'

In [5]:
#Define list of datas for each website
datas_matched_beer_names = ['df_matched_beer_beers','df_matched_beer_breweries','df_matched_beer_ratings_ba','df_matched_beer_ratings_rb','df_matched_beer_ratings','df_matched_beer_users_approx','df_matched_beer_users']
datas_advocate_names = ['df_advocate_beers','df_advocate_breweries','df_advocate_ratings','df_advocate_reviews','df_advocate_users']
datas_rate_beer_names = ['df_rate_beer_beers','df_rate_beer_breweries','df_rate_beer_users','df_rate_beer_ratings','df_rate_beer_ratings']

### Import datas stored as pickle

In [6]:
df_matched_beer_beers,df_matched_beer_breweries,df_matched_beer_ratings_ba,df_matched_beer_ratings_rb,df_matched_beer_ratings,df_matched_beer_users_approx,df_matched_beer_users = load_datas('matched_beer_data', datas_matched_beer_names)
df_rate_beer_beers,df_rate_beer_breweries,df_rate_beer_users,df_rate_beer_ratings,df_rate_beer_reviews = load_datas('RateBeer', datas_rate_beer_names)
df_advocate_beers,df_advocate_breweries,df_advocate_ratings,df_advocate_reviews,df_advocate_users = load_datas('BeerAdvocate', datas_advocate_names)

Loading df_matched_beer_beers...
Loading df_matched_beer_breweries...
Loading df_matched_beer_ratings_ba...
Loading df_matched_beer_ratings_rb...
Loading df_matched_beer_ratings...
Loading df_matched_beer_users_approx...
Loading df_matched_beer_users...
Loading df_rate_beer_beers...
Loading df_rate_beer_breweries...
Loading df_rate_beer_users...
Loading df_rate_beer_ratings...
Loading df_rate_beer_ratings...
Loading df_advocate_beers...
Loading df_advocate_breweries...
Loading df_advocate_ratings...
Loading df_advocate_reviews...
Loading df_advocate_users...


#### Cleaning

In [8]:
from cleaning_functions.matched_beer import *
from cleaning_functions.rate_beer import *
from cleaning_functions.advocate import *

In [9]:
mb_users = deepcopy(df_matched_beer_users)
users, mb_users_index = clean_mb_users(mb_users)
advocate_users = deepcopy(df_advocate_users)
advocate_users = clean_advocate_users(advocate_users)
rb_users = deepcopy(df_rate_beer_users)
rb_users = clean_rb_users(rb_users)

#### Merge

In [11]:
df_all_users = create_all_users(advocate_users,mb_users,rb_users)
df_ratings_merge = create_ratings_stat(df_advocate_ratings,df_rate_beer_ratings, df_all_users)

### Number of ratings per user

In [ ]:

mean_nbr_ratings_over_years=[]
median_nbr_ratings_over_years=[]
max_nbr_ratings_over_years=[]
quantile_5_nbr_ratings_over_years=[]
quantile_25_nbr_ratings_over_years=[]
quantile_75_nbr_ratings_over_years=[]


nbr_ratings_per_year_and_user=df_ratings_merge.groupby(["user_id","year"]).agg("size").reset_index()
years=np.sort(nbr_ratings_per_year_and_user.year.unique())
nbr_ratings_per_year_and_user=df_ratings_merge.groupby(["user_id","year"]).agg("size")

for a_year in years:
    mean_nbr_ratings_over_years.append(nbr_ratings_per_year_and_user[:,a_year].mean())
    median_nbr_ratings_over_years.append(nbr_ratings_per_year_and_user[:,a_year].median())
    quantile_5_nbr_ratings_over_years.append(nbr_ratings_per_year_and_user[:,a_year].quantile(0.95))
    quantile_25_nbr_ratings_over_years.append(nbr_ratings_per_year_and_user[:,a_year].quantile(0.25))
    quantile_75_nbr_ratings_over_years.append(nbr_ratings_per_year_and_user[:,a_year].quantile(0.75))
    max_nbr_ratings_over_years.append(nbr_ratings_per_year_and_user[:,a_year].max())
        
# Plot the results
plt.figure(figsize=(20,7))
markersize=3
plt.plot(years, max_nbr_ratings_over_years, label="max", c="m", linestyle="--", marker='o', markersize=markersize)
plt.plot(years, quantile_5_nbr_ratings_over_years, label="95th quantile", c="g", linestyle="--", marker='o', markersize=markersize)
plt.plot(years, quantile_75_nbr_ratings_over_years, label="75th quantile", c="k", linestyle="--", marker='o', markersize=markersize)
plt.plot(years, mean_nbr_ratings_over_years, label="mean", c="b", linestyle="--", marker='o', markersize=markersize)
plt.plot(years, median_nbr_ratings_over_years, label="median", c="r", linestyle="--", marker='o', markersize=markersize)
plt.plot(years, quantile_25_nbr_ratings_over_years, label="25th quantile", c="y", linestyle="--", marker='o', markersize=markersize)
plt.legend()
plt.yscale("log")
plt.ylabel("Number of ratings")
plt.xlabel("Years")
plt.title("Number of ratings per year and per user")
plt.show()
    

In [ ]:
# Now we plot the distribution of the number of ratings per user every year
fig,ax=plt.subplots(len(years)//3,3)
fig.set_size_inches(20,20)
for i,a_year in enumerate(years):
    ax[i//3,i%3].hist(nbr_ratings_per_year_and_user[:,a_year],bins=100,log=True)
    ax[i//3,i%3].axvline(nbr_ratings_per_year_and_user[:,a_year].quantile(0.95),c="r",linestyle="--")
    ax[i//3,i%3].axvline(nbr_ratings_per_year_and_user[:,a_year].quantile(0.99),c="orange",linestyle="--")

    ax[i//3,i%3].set_title(str(a_year))
    ax[i//3,i%3].set_xscale("log")
    ax[i//3,i%3].set_xlabel("Number of ratings")
    ax[i//3,i%3].set_ylabel("Number of users")
fig.tight_layout()
plt.show()

#### Now we need to add the cumulative number of ratings over the years for every beer

In [ ]:
sample_merge_ratings=df_ratings_merge.sample(100000,random_state=42)

In [ ]:
df_grouped_beer_year=df_ratings_merge.groupby(["beer_id","year"]).agg("size")

In [ ]:
sample_merge_ratings["nbr_ratings_beer_year-1"]=sample_merge_ratings[["beer_id","year"]].apply(lambda x: df_grouped_beer_year[str(x["beer_id"])][x["year"]-1] 
                                                if (x["year"]-1 in df_grouped_beer_year[str(x["beer_id"])])  else 0,axis=1)

In [ ]:
run_on_whole_ratings=False #might be long
if run_on_whole_ratings:
    df_ratings_merge["nbr_ratings_beer_year-1"]=df_ratings_merge[["beer_id","year"]].apply(lambda x: df_grouped_beer_year[str(x["beer_id"])][x["year"]-1] 
                                                    if (x["year"]-1 in df_grouped_beer_year[str(x["beer_id"])])  else 0,axis=1)

### Add experts column

In [4]:
from compute_experts import compute_experts_table,filter_year_and_add_is_expert

In [ ]:
experts_table=compute_experts_table(df_ratings_merge)

### Analyse a year (2016 for instance)

In [ ]:
df_ratings_this_year=filter_year_and_add_is_expert(df_ratings_merge,experts_table,2016)

In [ ]:
sample_df_ratings_this_year=filter_year_and_add_is_expert(sample_merge_ratings,experts_table,2016)

Number of ratings the year before the beer rated :

In [ ]:
nbr_ratings_year_minus1_experts=sample_df_ratings_this_year.loc[sample_df_ratings_this_year["is_expert"]==1]["nbr_ratings_beer_year-1"]
nbr_ratings_year_minus1_non_experts=sample_df_ratings_this_year.loc[sample_df_ratings_this_year["is_expert"]==0]["nbr_ratings_beer_year-1"]


In [ ]:
plt.hist(nbr_ratings_year_minus1_experts,bins=100,log=True,alpha=0.5,color="b",label="Experts")
plt.hist(nbr_ratings_year_minus1_non_experts,bins=100,log=True,alpha=0.5,color="orange",label="Non experts")
plt.legend()
plt.xlabel("Number of ratings of the beer the year before the rating")
plt.ylabel("Number of ratings")
plt.title(" Distribution of number of ratings of the beer the year before the rating, for experts and non experts")
plt.show()

In [ ]:
ttest_ind(nbr_ratings_year_minus1_experts,nbr_ratings_year_minus1_non_experts)

NOw goruped by user

In [ ]:
mean_nbr_ratings_years_minus_one_per_user=sample_df_ratings_this_year.groupby(["user_id","is_expert"])["nbr_ratings_beer_year-1"].agg("mean").reset_index().set_index("user_id")

In [ ]:
mean_nbr_ratings_years_minus_one_per_user_experts=mean_nbr_ratings_years_minus_one_per_user.loc[mean_nbr_ratings_years_minus_one_per_user["is_expert"]==1]["nbr_ratings_beer_year-1"]
mean_nbr_ratings_years_minus_one_per_user_non_experts=mean_nbr_ratings_years_minus_one_per_user.loc[mean_nbr_ratings_years_minus_one_per_user["is_expert"]==0]["nbr_ratings_beer_year-1"]

In [ ]:
plt.hist(mean_nbr_ratings_years_minus_one_per_user_experts,bins=100,log=True,alpha=0.5,color="b",label="Experts")
plt.hist(mean_nbr_ratings_years_minus_one_per_user_non_experts,bins=100,log=True,alpha=0.5,color="orange",label="Non experts")
plt.legend()
plt.xlabel("Mean number of ratings of the beer the year before the rating for a user")
plt.ylabel("Number of users")
plt.title(" Distribution of mean number of ratings of the beer the year before the rating,per user, for experts and non experts")
plt.show()

In [ ]:
ttest_ind(mean_nbr_ratings_years_minus_one_per_user_experts,mean_nbr_ratings_years_minus_one_per_user_non_experts)

#### Now let's analyse how their ratings differentiate

In [ ]:
minimum_number_of_ratings=100
YEAR=2017
absolute_difference_ratings=[]

df_ratings_this_year,experts_id=filter_year_and_add_is_expert(df_ratings_merge,YEAR,experts_table)
beers_with_enough_ratings=df_ratings_this_year.groupby("beer_id").agg("size").loc[df_ratings_this_year.groupby("beer_id").agg("size")>minimum_number_of_ratings].index.values
df_advocate_ratings_this_year_with_enough_ratings=df_ratings_this_year.loc[df_ratings_this_year["beer_id"].isin(beers_with_enough_ratings)]
difference_ratings_medians=df_advocate_ratings_this_year_with_enough_ratings.groupby(["beer_id","is_expert"])["rating"].agg("median")

for beer_id in difference_ratings_medians.index.get_level_values(0).unique():
    if 0 in difference_ratings_medians.loc[beer_id] and 1 in difference_ratings_medians.loc[beer_id]:
        absolute_difference_ratings.append(np.abs(difference_ratings_medians.loc[beer_id,0]-difference_ratings_medians.loc[beer_id,1]))


In [ ]:
plt.hist(absolute_difference_ratings,bins=50)
plt.xlabel("Absolute difference in median ratings")
plt.ylabel("Number of beers")
plt.show()

In [ ]:
YEAR=2015
df_ratings_this_year,experts_id=filter_year_and_add_is_expert(df_ratings_merge,YEAR,experts_table)


In [ ]:
fig,ax=plt.subplots(int(np.ceil(len(np.arange(50,500,50))/3)),3,sharex=True)
fig.set_size_inches(20,20)
nb_bins=20

for i,minimum_number_of_ratings in enumerate(np.arange(50,500,50)):
    print(minimum_number_of_ratings)
    difference_ratings_medians_list=[]
    difference_ratings_means_list=[]
    beers_with_enough_ratings=df_ratings_this_year.groupby("beer_id").agg("size").loc[df_ratings_this_year.groupby("beer_id").agg("size")>minimum_number_of_ratings].index.values
    df_advocate_ratings_this_year_with_enough_ratings=df_ratings_this_year.loc[df_ratings_this_year["beer_id"].isin(beers_with_enough_ratings)]
    difference_ratings_medians=df_advocate_ratings_this_year_with_enough_ratings.groupby(["beer_id","is_expert"])["rating"].agg("median")
    difference_ratings_means=df_advocate_ratings_this_year_with_enough_ratings.groupby(["beer_id","is_expert"])["rating"].agg("mean")
    
    for beer_id in difference_ratings_medians.index.get_level_values(0).unique():
        if 0 in difference_ratings_medians.loc[beer_id] and 1 in difference_ratings_medians.loc[beer_id]:
            difference_ratings_medians_list.append(difference_ratings_medians.loc[beer_id,1]-difference_ratings_medians.loc[beer_id,0])
            difference_ratings_means_list.append(difference_ratings_means.loc[beer_id,1]-difference_ratings_means.loc[beer_id,0])
    if i==0:
        difference_means_to_test=difference_ratings_means_list.copy()
    ax[i//3,i%3].hist(difference_ratings_medians_list,alpha=0.5,bins=nb_bins,color="b",label="median")
    ax[i//3,i%3].hist(difference_ratings_means_list,alpha=0.5,bins=nb_bins,color="orange",label="mean")
    ax[i//3,i%3].axvline(0,c="r",linestyle="--")
    ax[i//3,i%3].set_xlabel("difference in medians/means ratings (experts-non experts)")
    ax[i//3,i%3].set_ylabel("Number of beers")
    ax[i//3,i%3].set_title("Minimum number of ratings: "+str(minimum_number_of_ratings))
    ax[i//3,i%3].legend()
fig.suptitle("Distribution of difference of median and mean ratings of (experts - non experts) for different minimum number of ratings of beers",fontsize=15,y=1.02)
fig.tight_layout()
plt.show()

Now plot over the years

In [ ]:
fig,ax=plt.subplots(4,3,sharex=True)
fig.set_size_inches(20,20)
nb_bins=20
min_number_of_ratings=50
quantile_to_be_expert=0.95
for i,YEAR in enumerate(range(2006,2018)):
    print(YEAR)
    difference_ratings_medians_list=[]
    difference_ratings_means_list=[]
    df_ratings_this_year,experts_id=filter_year_and_add_is_expert(df_ratings_merge,YEAR,experts_table)
    beers_with_enough_ratings=df_ratings_this_year.groupby("beer_id").agg("size").loc[df_ratings_this_year.groupby("beer_id").agg("size")>min_number_of_ratings].index.values
    df_advocate_ratings_this_year_with_enough_ratings=df_ratings_this_year.loc[df_ratings_this_year["beer_id"].isin(beers_with_enough_ratings)]
    difference_ratings_medians=df_advocate_ratings_this_year_with_enough_ratings.groupby(["beer_id","is_expert"])["rating"].agg("median")
    difference_ratings_means=df_advocate_ratings_this_year_with_enough_ratings.groupby(["beer_id","is_expert"])["rating"].agg("mean")
    
    for beer_id in difference_ratings_medians.index.get_level_values(0).unique():
        if 0 in difference_ratings_medians.loc[beer_id] and 1 in difference_ratings_medians.loc[beer_id]:
            difference_ratings_medians_list.append(difference_ratings_medians.loc[beer_id,1]-difference_ratings_medians.loc[beer_id,0])
            difference_ratings_means_list.append(difference_ratings_means.loc[beer_id,1]-difference_ratings_means.loc[beer_id,0])
            
    ax[i//3,i%3].hist(difference_ratings_medians_list,alpha=0.5,bins=nb_bins,color="b",label="median")
    ax[i//3,i%3].hist(difference_ratings_means_list,alpha=0.5,bins=nb_bins,color="orange",label="mean")
    ax[i//3,i%3].axvline(0,c="r",linestyle="--")
    ax[i//3,i%3].set_xlim(-1.25,1)
    ax[i//3,i%3].set_xlabel("Absolute difference in median ratings (experts-non experts)")
    ax[i//3,i%3].set_ylabel("Number of beers")
    ax[i//3,i%3].set_title("Year: "+str(YEAR))
    ax[i//3,i%3].legend()
fig.suptitle(f"Distribution of difference of median and mean ratings of (experts - non experts) for different years and minimum number of ratings of beers: {min_number_of_ratings}",fontsize=15,y=1.02)
fig.tight_layout()
plt.show()


#### Now analysis beer by beer

In [ ]:
YEAR=2016
df_ratings_this_year,experts_id=filter_year_and_add_is_expert(df_ratings_merge,YEAR,experts_table)

In [ ]:
fig,ax=plt.subplots(5,5)
fig.set_size_inches(20,20)
minimum_number_of_ratings=200
bins=10
for i in range(25):
    beer_id_to_study=np.random.choice(df_ratings_this_year.groupby("beer_id").agg("size").loc[df_ratings_this_year.groupby("beer_id").agg("size")>minimum_number_of_ratings].index.values)
    ratings_this_beer=df_ratings_this_year.loc[df_ratings_this_year["beer_id"]==beer_id_to_study]
    ratings_this_beer_experts=ratings_this_beer.loc[ratings_this_beer["is_expert"]==1]
    ratings_this_beer_non_experts=ratings_this_beer.loc[ratings_this_beer["is_expert"]==0]
    
    difference_ttest=ttest_ind(ratings_this_beer_experts["rating"],ratings_this_beer_non_experts["rating"])
    ax[i//5,i%5].hist(ratings_this_beer_experts["rating"],bins=bins,alpha=0.5,label="Experts",color="b")
    ax[i//5,i%5].hist(ratings_this_beer_non_experts["rating"],bins=bins,alpha=0.5,label="Non experts",color="orange")
    ax[i//5,i%5].axvline(ratings_this_beer_experts["rating"].mean(),c="b",linestyle="--")
    ax[i//5,i%5].axvline(ratings_this_beer_non_experts["rating"].mean(),c="orange",linestyle="--")
    
    ax[i//5,i%5].set_xticks(np.arange(0,6,1),labels=np.arange(0,6,1))
    ax[i//5,i%5].set_xlabel("Rating")
    ax[i//5,i%5].set_ylabel("Number of ratings")
    ax[i//5, i%5].set_title(f"Beer id: {beer_id_to_study} Nbr of ratings: {ratings_this_beer.shape[0]}\n ttest diff stat: {np.round(difference_ttest[0], 2)}, pvalue: {np.round(difference_ttest[1], 2)}")
for i in range(5):
    ax[i,0].legend()
fig.suptitle(f"Distribution of some ratings for experts and non experts for year {YEAR}",y=1.01,fontsize=20)
fig.tight_layout()
plt.show()

#### Beers with significant diff:

In [ ]:
relaunch_search=False
if relaunch_search:
    beer_id_with_significant_difference=[]
    minimum_number_of_ratings=10
    for beer_id_to_study in df_ratings_this_year.groupby("beer_id").agg("size").loc[df_ratings_this_year.groupby("beer_id").agg("size")>minimum_number_of_ratings].index.values:
        ratings_this_beer=df_ratings_this_year.loc[df_ratings_this_year["beer_id"]==beer_id_to_study]
        ratings_this_beer_experts=ratings_this_beer.loc[ratings_this_beer["is_expert"]==1]
        ratings_this_beer_non_experts=ratings_this_beer.loc[ratings_this_beer["is_expert"]==0]
        
        difference_ttest=ttest_ind(ratings_this_beer_experts["rating"],ratings_this_beer_non_experts["rating"])
        if difference_ttest[1]<0.05:
            beer_id_with_significant_difference.append(beer_id_to_study)
else:
    with open("intermediate_files/beer_id_with_significant_difference_2015_0.95.pkl","rb") as f:
        beer_id_with_significant_difference=pickle.load(f)

In [ ]:
fig,ax=plt.subplots(5,5)
fig.set_size_inches(20,20)

bins=10
for i in range(25):
    beer_id_to_study=np.random.choice(beer_id_with_significant_difference)
    ratings_this_beer=df_ratings_this_year.loc[df_ratings_this_year["beer_id"]==beer_id_to_study]
    ratings_this_beer_experts=ratings_this_beer.loc[ratings_this_beer["is_expert"]==1]
    ratings_this_beer_non_experts=ratings_this_beer.loc[ratings_this_beer["is_expert"]==0]
    
    difference_ttest=ttest_ind(ratings_this_beer_experts["rating"],ratings_this_beer_non_experts["rating"])
    ax[i//5,i%5].hist(ratings_this_beer_experts["rating"],bins=bins,alpha=0.5,label="Experts",color="b")
    ax[i//5,i%5].axvline(ratings_this_beer_experts["rating"].mean(),c="b",linestyle="--")
    ax[i//5,i%5].axvline(ratings_this_beer_non_experts["rating"].mean(),c="orange",linestyle="--")
    ax[i//5,i%5].hist(ratings_this_beer_non_experts["rating"],bins=bins,alpha=0.5,label="Non experts",color="orange")
    
    ax[i//5,i%5].set_xticks(np.arange(0,6,1),labels=np.arange(0,6,1))
    ax[i//5,i%5].set_xlabel("Rating")
    ax[i//5,i%5].set_ylabel("Number of ratings")
    ax[i//5, i%5].set_title(f"Beer id: {beer_id_to_study} Nbr of ratings: {ratings_this_beer.shape[0]}\n ttest diff stat: {np.round(difference_ttest[0], 2)}, pvalue: {np.round(difference_ttest[1], 4)}")
for i in range(5):
    ax[i,0].legend()
fig.suptitle(f"Distribution of some ratings for experts and non experts for year {YEAR}",y=1.01,fontsize=20)
fig.tight_layout()
plt.show()